In [11]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import re

from bs4 import BeautifulSoup
from utils import *

path_2004 = "./test/"
all_2004 = os.listdir(path_2004)
all_2004 = [x for x in all_2004 if ('DS_Store' not in x)]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
results = []
for i, fname in enumerate(all_2004):
    print("File: ", i)
    test_f = open(path_2004 + fname)
    wholetext = test_f.read()
    cikbase = re.search(r"CENTRAL INDEX KEY:\s*?\d{10}\b", wholetext, re.I).group()
    cik_re = re.split(r'\s', cikbase)[-1]        

    datadatebase = re.search(r"CONFORMED PERIOD OF REPORT:\s*?\d{8}\b", wholetext, re.I).group() 
    datadate_re = re.split(r'\s', datadatebase)[-1]  

    wholetext = wholetext.lower()
    tbl_flag = 0 # 1 if "contractual obligations" table exists
    row_flag = 0 # 1 if "purchasing obligations" row exists
    multiplier = 1 # 1 by default
    
    table = scrape_table(wholetext)
    if table:
        tbl_flag = 1
        rows = scrape_rows(table)
        multiplier = get_multiplier_from_tbl(table)
    else:
        lines, tbl_list = scrape_text_table(wholetext)
        if tbl_list:
            tbl_flag = 1
            rows = scrape_text_rows(lines, tbl_list)
            multiplier = get_multiplier_from_tbl_list(tbl_list)

            
    for row in rows:
        row_flag = 1
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                   '<1': row['<1'], '1-3':row['1-3'], '3-5':row['3-5'], '>5':row['>5'], 
                  'category': row['category'], 'multiplier': multiplier}
        if 'total' in row.keys():
                result['total'] = row['total']
        results.append(result)

    # No table or no row
    if (tbl_flag == 0) or (row_flag == 0):
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                  'total': '-', '<1': '-', '1-3':'-', '3-5':'-', '>5':'-', 'multiplier': multiplier}

        results.append(result)
pd.DataFrame(results).to_csv('scraped_files_df')

File:  0


/Users/divyas/Documents/G1/sec/utils.py:152: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 152 of the file /Users/divyas/Documents/G1/sec/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  parsed_html = BeautifulSoup(wholetext)


Found a table
File:  1
Found a table
File:  2
Found a table
File:  3
Found a table
File:  4
Found a table
File:  5
Found a table
Unknown error:  could not convert string to float: 'thereafter total minimum lease payments'
File:  6
Found a table
File:  7
Found a table
File:  8
Found a table
File:  9
Found a table
File:  10
Found a table
File:  11
Found a table
File:  12
Found a table
File:  13
Found a table
File:  14
Found a table
File:  15
Found a table
File:  16
Found a table
File:  17
Found a table
File:  18
Found a table
File:  19
Found a table
File:  20
Found a table
File:  21
Found a table
File:  22
Found a table
File:  23
Found a table
File:  24
Found a table
File:  25
Found a table
File:  26
Found a table
File:  27
Found a table
File:  28
Found a table
File:  29
Found a table
File:  30
Found a table
File:  31
Found a table
File:  32
Found a table
File:  33
Found a table
File:  34
Found a table
File:  35
Found a table
File:  36
Found a table
File:  37
Found a table
File:  38
Foun

In [3]:
results_df = pd.DataFrame(results)
row_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 1)] # table but no row
none_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 0)] # no table no row
last_check = results_df[(results_df['row'] == 1) & (results_df['tbl'] == 1)] # table and row, other issue?
print(len(set(last_check['fname'])), len(set(row_check['fname'])), len(set(none_check['fname'])))

73 18 0


In [6]:
last_check

,1-3,3-5,<1,>5,category,cik,datadate,fname,multiplier,row,tbl,total
0,146,2,136,3,capital lease payments (2),0000065984,20041231,0000065984-05-000147.txt,1000000,1,1,287
1,155,100,99,210,operating leases (2),0000065984,20041231,0000065984-05-000147.txt,1000000,1,1,564
2,1402,962,1160,1156,purchase obligations (3),0000065984,20041231,0000065984-05-000147.txt,1000000,1,1,4680
3,0.4,0.1,0.5,0,capital\n lease obligations (c),0000004904,20041231,0001015402-05-001007.txt,1,1,1,1
4,7.6,5.1,5.8,6.2,noncancelable\n operating leases (c),0000004904,20041231,0001015402-05-001007.txt,1,1,1,24.7
5,46.1,41.8,22.9,96.7,energy\n and capacity purchase contracts (d),0000004904,20041231,0001015402-05-001007.txt,1,1,1,207.5
6,5446,3539,2620,482,capital lease obligations,0001056904,20041231,0001193125-05-032959.txt,1000,1,1,12087
7,13941,10742,7633,19539,operating leases,0001056904,20041231,0001193125-05-032959.txt,1000,1,1,51855
8,234,319,93,976,operating leases,0000879215,20041231,0001193125-05-048164.txt,1,1,1,1622
9,385,401,208,1248,capacity payments,0000879215,20041231,0001193125-05-048164.txt,1,1,1,2242


# Examining one file

In [27]:
test_f = open(path_2004 + "0000003753-03-000009.txt")
wholetext = test_f.read()
cik_re = get_cik(wholetext)
datadate_re = get_datadate(wholetext)  
print("CIK: ", cik_re, "Date: ", datadate_re)
wholetext = wholetext.lower()
lines, tbl_list = scrape_text_table(wholetext)

CIK:  0000003753 Date:  20021231


In [20]:
rows = []
for row in tbl_list:
    if any([word in row[0] for word in ROW_KWS]):
        if ('following' not in row[0]):
            rows.append(clean_row(row))


In [28]:
scrape_text_rows(lines, tbl_list)

[{'1-3': '555750',
  '3-5': '0',
  '<1': '312856',
  '>5': '0',
  'category': 'operating leases',
  'total': 868606.0}]